In [1]:
import nltk
import json
import pickle
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
import numpy as np
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential
from keras.optimizers import SGD
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
#loading json file
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenizing each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #adding documents in corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words
print (len(words), "unique lemmatized words", words)

135 documents
21 classes ['admission', 'bba', 'bba course', 'bit', 'bit course', 'ceo', 'contact', 'courses', 'established', 'facilities', 'goodbye', 'greeting', 'hours', 'location', 'login', 'mba', 'mba course', 'name', 'options', 'register', 'scholarship']
101 unique lemmatized words ["'s", '.', 'a', 'address', 'admission', 'am', 'any', 'anyone', 'are', 'at', 'available', 'bba', 'be', 'bit', 'bye', 'call', 'can', 'ceo', 'college', 'contact', 'could', 'course', 'cya', 'date', 'day', 'department', 'detail', 'do', 'doe', 'established', 'facilites', 'facility', 'founded', 'good', 'goodbye', 'guide', 'have', 'hello', 'help', 'helpful', 'hi', 'hii', 'hour', 'how', 'i', 'in', 'is', 'it', 'later', 'leaving', 'located', 'location', 'log', 'login', 'many', 'mba', 'me', 'name', 'new', 'number', 'of', 'offered', 'ok', 'open', 'operation', 'option', 'phone', 'phonenumber', 'provide', 'provided', 'reg', 'register', 'scholarship', 'see', 'service', 'session', 'should', 'signin', 'signup', 'start', 

In [6]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [8]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-8-9ffac2c3f218>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [9]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [10]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [11]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)
print("model created")

Epoch 1/200
27/27 [==============================] - 0s 959us/step - loss: 3.0946 - accuracy: 0.0356
Epoch 2/200
27/27 [==============================] - 0s 921us/step - loss: 2.9761 - accuracy: 0.1311
Epoch 3/200
27/27 [==============================] - 0s 958us/step - loss: 2.9072 - accuracy: 0.1478
Epoch 4/200
27/27 [==============================] - 0s 921us/step - loss: 2.8661 - accuracy: 0.1192
Epoch 5/200
27/27 [==============================] - 0s 844us/step - loss: 2.7460 - accuracy: 0.2432
Epoch 6/200
27/27 [==============================] - 0s 725us/step - loss: 2.5908 - accuracy: 0.2915
Epoch 7/200
27/27 [==============================] - 0s 645us/step - loss: 2.5258 - accuracy: 0.2507
Epoch 8/200
27/27 [==============================] - 0s 729us/step - loss: 2.3073 - accuracy: 0.3475
Epoch 9/200
27/27 [==============================] - 0s 760us/step - loss: 2.0289 - accuracy: 0.3979
Epoch 10/200
27/27 [==============================] - 0s 729us/step - loss: 1.9748 - accura

Epoch 81/200
27/27 [==============================] - 0s 730us/step - loss: 0.1085 - accuracy: 0.9969
Epoch 82/200
27/27 [==============================] - 0s 766us/step - loss: 0.0899 - accuracy: 0.9809
Epoch 83/200
27/27 [==============================] - 0s 728us/step - loss: 0.0899 - accuracy: 0.9827
Epoch 84/200
27/27 [==============================] - 0s 691us/step - loss: 0.1710 - accuracy: 0.9640
Epoch 85/200
27/27 [==============================] - 0s 652us/step - loss: 0.1004 - accuracy: 0.9746
Epoch 86/200
27/27 [==============================] - 0s 575us/step - loss: 0.0984 - accuracy: 0.9862
Epoch 87/200
27/27 [==============================] - 0s 729us/step - loss: 0.1017 - accuracy: 0.9612
Epoch 88/200
27/27 [==============================] - 0s 651us/step - loss: 0.0729 - accuracy: 0.9796
Epoch 89/200
27/27 [==============================] - 0s 728us/step - loss: 0.1903 - accuracy: 0.9444
Epoch 90/200
27/27 [==============================] - 0s 728us/step - loss: 0.0820

27/27 [==============================] - 0s 844us/step - loss: 0.0391 - accuracy: 1.0000
Epoch 161/200
27/27 [==============================] - 0s 805us/step - loss: 0.0259 - accuracy: 1.0000
Epoch 162/200
27/27 [==============================] - 0s 844us/step - loss: 0.0290 - accuracy: 1.0000
Epoch 163/200
27/27 [==============================] - 0s 833us/step - loss: 0.0609 - accuracy: 0.9884
Epoch 164/200
27/27 [==============================] - 0s 769us/step - loss: 0.0392 - accuracy: 0.9877
Epoch 165/200
27/27 [==============================] - 0s 767us/step - loss: 0.0879 - accuracy: 0.9595
Epoch 166/200
27/27 [==============================] - 0s 813us/step - loss: 0.0519 - accuracy: 0.9924
Epoch 167/200
27/27 [==============================] - 0s 806us/step - loss: 0.0185 - accuracy: 0.9980
Epoch 168/200
27/27 [==============================] - 0s 806us/step - loss: 0.0861 - accuracy: 0.9832
Epoch 169/200
27/27 [==============================] - 0s 768us/step - loss: 0.0370 - a